In [1]:
!pip install torch torchtext transformers sentencepiece pandas tqdm datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 21.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 21.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 10.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [2]:
from datasets import load_dataset, DatasetDict, Dataset
import pandas as pd
import ast
import datasets
from tqdm import tqdm
import time


In [3]:

#Load Dataset
data_sample=load_dataset('QuyenAnhDE/Diseases_Symptoms')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/381 [00:00<?, ?B/s]

Repo card metadata block was not found. Setting CardData to empty.


Diseases_Symptoms.csv:   0%|          | 0.00/107k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/400 [00:00<?, ? examples/s]

In [4]:
data_sample

DatasetDict({
    train: Dataset({
        features: ['Code', 'Name', 'Symptoms', 'Treatments'],
        num_rows: 400
    })
})

In [5]:
updated_data=[{'Name':item['Name'],'Symptoms':item['Symptoms']} for item in data_sample['train']]

In [6]:
df=pd.DataFrame(updated_data)

In [7]:
df.head()

,Name,Symptoms
0,Panic disorder,"Palpitations, Sweating, Trembling, Shortness o..."
1,Vocal cord polyp,"Hoarseness, Vocal Changes, Vocal Fatigue"
2,Turner syndrome,"Short stature, Gonadal dysgenesis, Webbed neck..."
3,Cryptorchidism,"Absence or undescended testicle(s), empty scro..."
4,Ethylene glycol poisoning-1,"Nausea, vomiting, abdominal pain, General mala..."


In [8]:
#Extract the symptoms
df['Symptoms']=df['Symptoms'].apply(lambda x:','.join(x.split(',')))

In [9]:
df['Symptoms']

,Symptoms
0,"Palpitations, Sweating, Trembling, Shortness o..."
1,"Hoarseness, Vocal Changes, Vocal Fatigue"
2,"Short stature, Gonadal dysgenesis, Webbed neck..."
3,"Absence or undescended testicle(s), empty scro..."
4,"Nausea, vomiting, abdominal pain, General mala..."
...,...
395,"Severe abdominal or back pain, blood in urine,..."
396,"Fragile bones, loss of height over time, back ..."
397,"Joint pain, stiffness, swelling, fatigue, loss..."
398,"Frequent urination, Increased thirst, Weight loss"


In [10]:
from transformers import GPT2Tokenizer,GPT2LMHeadModel
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader,random_split


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [11]:
# If you have an NVIDIA GPU attached, use 'cuda'
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    # If Apple Silicon, set to 'mps' - otherwise 'cpu' (not advised)
    try:
        device = torch.device('mps')
    except Exception:
        device = torch.device('cpu')

In [12]:
device

device(type='cuda')

In [43]:
tokenizer=GPT2Tokenizer.from_pretrained('distilgpt2')
model=GPT2LMHeadModel.from_pretrained('distilgpt2').to(device)

In [44]:
model

GPT2LMHeadModel(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 768)
    (wpe): Embedding(1024, 768)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2SdpaAttention(
          (c_attn): Conv1D(nf=2304, nx=768)
          (c_proj): Conv1D(nf=768, nx=768)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=3072, nx=768)
          (c_proj): Conv1D(nf=768, nx=3072)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
  )
  (lm_head): Linear(in_features=768, out_features=50257, bias=False)
)

In [45]:
BATCH_SIZE=8

In [46]:
df.describe()

,Name,Symptoms
count,400,400
unique,392,395
top,Sciatica,"Swelling, pain, dry mouth, bad taste"
freq,3,3


In [47]:
#Dataset prep
class LanguageDataset(Dataset):
    def __init__(self, df, tokenizer):
        self.labels = df.columns
        self.data=df.to_dict(orient='records')
        self.tokenizer = tokenizer
        x=self.fittest_max_length(df)
        self.max_length=x

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x=self.data[idx][self.labels[0]]
        y=self.data[idx][self.labels[1]]
        text=f"{x}|{y}"
        tokens=self.tokenizer.encode_plus(text,return_tensors='pt',padding='max_length',max_length=128,truncation=True)
        return tokens

    def fittest_max_length(self,df):
        max_length=max(len(max(df[self.labels[0]],key=len)),len(max(df[self.labels[1]],key=len)))
        x=2
        while x<max_length:x=x*2
        return x

In [48]:
data_sample=LanguageDataset(df,tokenizer)

In [49]:
data_sample

In [50]:
train_size=int(0.8*len(data_sample))
val_size=len(data_sample)-train_size
train_data,val_data=random_split(data_sample,[train_size,val_size])


In [51]:
print(len(train_data),len(val_data))

320 80


In [52]:
train_loader=DataLoader(train_data,batch_size=BATCH_SIZE,shuffle=True)
val_loader=DataLoader(val_data,batch_size=BATCH_SIZE)

In [53]:
num_epochs=8

In [54]:
batch_size=BATCH_SIZE
model_name='distilgpt2'
gpu=0

In [55]:

# Set the learning rate and loss function
## CrossEntropyLoss measures how close answers to the truth.
## More punishing for high confidence wrong answers
criterion = nn.CrossEntropyLoss(ignore_index = tokenizer.pad_token_id)
optimizer = optim.Adam(model.parameters(), lr=5e-4)
tokenizer.pad_token = tokenizer.eos_token


In [57]:
# Init a results dataframe
results = pd.DataFrame(columns=['epoch', 'transformer', 'batch_size', 'gpu',
                                'training_loss', 'validation_loss', 'epoch_duration_sec'])


In [59]:
# The training loop
for epoch in range(num_epochs):
    start_time = time.time()  # Start the timer for the epoch

    # Training
    ## This line tells the model we're in 'learning mode'
    model.train()
    epoch_training_loss = 0
    train_iterator = tqdm(train_loader, desc=f"Training Epoch {epoch+1}/{num_epochs} Batch Size: {batch_size}, Transformer: {model_name}")
    for batch in train_iterator:
        optimizer.zero_grad()
        inputs = batch['input_ids'].squeeze(1).to(device)
        targets = inputs.clone()
        outputs = model(input_ids=inputs, labels=targets)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        train_iterator.set_postfix({'Training Loss': loss.item()})
        epoch_training_loss += loss.item()
    avg_epoch_training_loss = epoch_training_loss / len(train_iterator)

    # Validation
    ## This line below tells the model to 'stop learning'
    model.eval()
    epoch_validation_loss = 0
    total_loss = 0
    # The fix: Use val_loader instead of valid_loader
    valid_iterator = tqdm(val_loader, desc=f"Validation Epoch {epoch+1}/{num_epochs}")
    with torch.no_grad():
        for batch in valid_iterator:
            inputs = batch['input_ids'].squeeze(1).to(device)
            targets = inputs.clone()
            outputs = model(input_ids=inputs, labels=targets)
            loss = outputs.loss
            total_loss += loss
            valid_iterator.set_postfix({'Validation Loss': loss.item()})
            epoch_validation_loss += loss.item()

    avg_epoch_validation_loss = epoch_validation_loss / len(val_loader) #The fix: Use val_loader instead of valid_loader

    end_time = time.time()  # End the timer for the epoch
    epoch_duration_sec = end_time - start_time  # Calculate the duration in seconds

    new_row = {'transformer': model_name,
               'batch_size': batch_size,
               'gpu': gpu,
               'epoch': epoch+1,
               'training_loss': avg_epoch_training_loss,
               'validation_loss': avg_epoch_validation_loss,
               'epoch_duration_sec': epoch_duration_sec}  # Add epoch_duration to the dataframe

    results.loc[len(results)] = new_row
    print(f"Epoch: {epoch+1}, Validation Loss: {total_loss/len(val_loader)}") #The fix: Use val_loader instead of valid_loader

Training Epoch 1/8 Batch Size: 8, Transformer: distilgpt2: 100%|██████████| 40/40 [00:07<00:00,  5.36it/s, Training Loss=0.501]
Validation Epoch 1/8: 100%|██████████| 10/10 [00:00<00:00, 17.92it/s, Validation Loss=0.716]


Epoch: 1, Validation Loss: 0.6770339608192444


Training Epoch 2/8 Batch Size: 8, Transformer: distilgpt2: 100%|██████████| 40/40 [00:07<00:00,  5.41it/s, Training Loss=0.3]
Validation Epoch 2/8: 100%|██████████| 10/10 [00:00<00:00, 18.30it/s, Validation Loss=0.764]


Epoch: 2, Validation Loss: 0.7009806632995605


Training Epoch 3/8 Batch Size: 8, Transformer: distilgpt2: 100%|██████████| 40/40 [00:07<00:00,  5.34it/s, Training Loss=0.348]
Validation Epoch 3/8: 100%|██████████| 10/10 [00:00<00:00, 18.01it/s, Validation Loss=0.808]


...

Epoch: 6, Validation Loss: 0.879849910736084


Training Epoch 7/8 Batch Size: 8, Transformer: distilgpt2: 100%|██████████| 40/40 [00:07<00:00,  5.10it/s, Training Loss=0.128]
Validation Epoch 7/8: 100%|██████████| 10/10 [00:00<00:00, 17.21it/s, Validation Loss=1.06]


Epoch: 7, Validation Loss: 0.9370619654655457


Training Epoch 8/8 Batch Size: 8, Transformer: distilgpt2: 100%|██████████| 40/40 [00:07<00:00,  5.24it/s, Training Loss=0.099]
Validation Epoch 8/8: 100%|██████████| 10/10 [00:00<00:00, 17.96it/s, Validation Loss=1.08]

Epoch: 8, Validation Loss: 0.9637420773506165


In [60]:
input_str = "Kidney Failure"
input_ids = tokenizer.encode(input_str, return_tensors='pt').to(device)

output = model.generate(
    input_ids,
    max_length=20,
    num_return_sequences=1,
    do_sample=True,
    top_k=8,
    top_p=0.95,
    temperature=0.5,
    repetition_penalty=1.2
)

decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
print(decoded_output)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Kidney Failure|Decreased urine output, fluid retention, fatigue, shortness of breath


In [61]:
torch.save(model, 'SmallMedLM.pt')

In [63]:
#torch.save(model, 'drive/My Drive/SmallMedLM.pt')